<a href="https://colab.research.google.com/github/sanyhmahadi/Fake-News-Detection/blob/master/fake_Headlines_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
df=pd.read_excel("/content/drive/MyDrive/Research/NLP/Sentiment Data/FakevsAuthentic news edit sany.xlsx")

In [ ]:
df.shape

(6193, 2)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import itertools
from sklearn.model_selection import train_test_split
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import string as st
import re
import nltk
from nltk import PorterStemmer, WordNetLemmatizer
import matplotlib.pyplot as plt

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))


In [ ]:
df.head()

,Headline,Label
0,মুরগির হামলায় শেয়াল নিহত,Fake
1,বিটিভিতে যেবার আমি ইন্টারভিউ দিতে গেলাম,Fake
2,বিদেশ থেকে উন্নতমানের বিরোধীদল আমদানি করার পরা...,Fake
3,অবসর নেয়ার ঘোষণা দিলেন মেসি !,Fake
4,"মাদারফাকার নহে, ব্রাদারফাকার: সাকা | দৈনিক মতি...",Fake


In [ ]:
print(np.unique(df['Label']))
print(np.unique(df['Label'].value_counts()))

['Authentic' 'Fake']
[3095 3098]


In [ ]:
df['Headline'].value_counts()

এক যে আছে মজার দেশ সব রকমে ভালো, পরের মুখে হাঁচি দিতে সবার লাগে ভালো!    10
ভাঙা গাল শেভ করার যে দুর্দান্ত টেকনিক আপনার অবশ্যই শিখে নেয়া উচিত         8
একজন নারী দেহরক্ষীর গোপন জীবন                                             6
কবির দুচোখ সাদা, তাতে এক ফোঁটা নেই লালও!                                  4
দ্রুত ডেলিভারি দিতে ইন্টারনেট এক্সপ্লোরার কিনতে চায় ইভ্যালি               4
                                                                         ..
অর্থপাচার: ২১ দফা অভিযোগের মুখে নাজিব রাজাক                               1
'অাদালতে হাজির হওয়ার মতো শারীরিক সুস্থতা নেই খালেদা জিয়ার'                1
বন্দরে সেফটিক ট্যাংকে বিস্ফোরণে ভবনের ব্যাপক ক্ষতি, আহত ৪                 1
বিদেশে “বিরল অর্জন” দেশে বিরাট সংবর্ধনা! | দৈনিক মতিকণ্ঠ                  1
৩০০ আসনে প্রার্থী দিতে প্রস্তুতি নিচ্ছে বৃহত্তর জাতীয় ঐক্য                1
Name: Headline, Length: 5903, dtype: int64

In [ ]:
def remove_punct(News):
    return ("".join([ch for ch in News if ch not in st.punctuation]))

In [ ]:
df['removed_punc'] = df['Headline'].apply(lambda x: remove_punct(x))
df.head()

,Headline,Label,removed_punc
0,মুরগির হামলায় শেয়াল নিহত,Fake,মুরগির হামলায় শেয়াল নিহত
1,বিটিভিতে যেবার আমি ইন্টারভিউ দিতে গেলাম,Fake,বিটিভিতে যেবার আমি ইন্টারভিউ দিতে গেলাম
2,বিদেশ থেকে উন্নতমানের বিরোধীদল আমদানি করার পরা...,Fake,বিদেশ থেকে উন্নতমানের বিরোধীদল আমদানি করার পরা...
3,অবসর নেয়ার ঘোষণা দিলেন মেসি !,Fake,অবসর নেয়ার ঘোষণা দিলেন মেসি
4,"মাদারফাকার নহে, ব্রাদারফাকার: সাকা | দৈনিক মতি...",Fake,মাদারফাকার নহে ব্রাদারফাকার সাকা দৈনিক মতিকণ্ঠ


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df.head(5)

,Headline,Label,removed_punc
0,মুরগির হামলায় শেয়াল নিহত,Fake,মুরগির হামলায় শেয়াল নিহত
1,বিটিভিতে যেবার আমি ইন্টারভিউ দিতে গেলাম,Fake,বিটিভিতে যেবার আমি ইন্টারভিউ দিতে গেলাম
2,বিদেশ থেকে উন্নতমানের বিরোধীদল আমদানি করার পরা...,Fake,বিদেশ থেকে উন্নতমানের বিরোধীদল আমদানি করার পরা...
3,অবসর নেয়ার ঘোষণা দিলেন মেসি !,Fake,অবসর নেয়ার ঘোষণা দিলেন মেসি
4,"মাদারফাকার নহে, ব্রাদারফাকার: সাকা | দৈনিক মতি...",Fake,মাদারফাকার নহে ব্রাদারফাকার সাকা দৈনিক মতিকণ্ঠ


In [ ]:
y = df.Label 
df.drop("Label", axis=1)

,Headline,removed_punc
0,মুরগির হামলায় শেয়াল নিহত,মুরগির হামলায় শেয়াল নিহত
1,বিটিভিতে যেবার আমি ইন্টারভিউ দিতে গেলাম,বিটিভিতে যেবার আমি ইন্টারভিউ দিতে গেলাম
2,বিদেশ থেকে উন্নতমানের বিরোধীদল আমদানি করার পরা...,বিদেশ থেকে উন্নতমানের বিরোধীদল আমদানি করার পরা...
3,অবসর নেয়ার ঘোষণা দিলেন মেসি !,অবসর নেয়ার ঘোষণা দিলেন মেসি
4,"মাদারফাকার নহে, ব্রাদারফাকার: সাকা | দৈনিক মতি...",মাদারফাকার নহে ব্রাদারফাকার সাকা দৈনিক মতিকণ্ঠ
...,...,...
6188,সিএমপির তিন থানার ওসি পদে রদবদল,সিএমপির তিন থানার ওসি পদে রদবদল
6189,বল হাতে উইকেটে জাদু দেখাচ্ছেন সাকিব,বল হাতে উইকেটে জাদু দেখাচ্ছেন সাকিব
6190,রোহিঙ্গা সংকট নিরসনে রাজনৈতিক সমাধানের আহ্বান,রোহিঙ্গা সংকট নিরসনে রাজনৈতিক সমাধানের আহ্বান
6191,অস্বাভাবিক দাম বৃদ্ধির কারণ জানে না চার কোম্পানি,অস্বাভাবিক দাম বৃদ্ধির কারণ জানে না চার কোম্পানি


In [ ]:
df.Headline = df.Headline.str.replace(r'http[\w:/\.]+','<URL>') 

df.Headline = df.Headline.str.replace(r'[^\.\w\s]','') 
df.Headline = df.Headline.str.replace(r'\.\.+','.') 

df.Headline = df.Headline.str.replace(r'\.',' . ') 

df.Headline = df.Headline.str.replace(r'\s\s+',' ') 

df.Headline = df.Headline.str.strip()

In [ ]:
df.Headline.values

array(['মরগর হমলয় শয়ল নহত', 'বটভত যবর আম ইনটরভউ দত গলম',
       'বদশ থক উননতমনর বরধদল আমদন করর পরমরশ দলন অনলইন রজনত বশষজঞর', ...,
       'রহঙগ সকট নরসন রজনতক সমধনর আহবন',
       'অসবভবক দম বদধর করণ জন ন চর কমপন',
       'শশবর সতত শকষ শকষরথদর সনগরক করব'], dtype=object)

In [ ]:
from collections import Counter
all_text = ' '.join(str(v) for v in df.Headline.values)
words = all_text.split()
u_words = Counter(words).most_common()
u_words_counter = u_words
u_words_frequent = [word[0] for word in u_words if word[1]>5]

In [ ]:
import matplotlib.pyplot as plt

import re

from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, LSTM, Embedding, Reshape
from keras.models import load_model, model_from_json

from sklearn.model_selection import train_test_split

import os
import urllib

from urllib.request import urlretrieve

from os import mkdir, makedirs, remove, listdir

from collections import Counter


with open('/content/drive/MyDrive/Research/NLP/GloveTexFiles/bn_glove.39M.300d.txt','rb') as f:
    lines = f.readlines()
    
glove_weights = np.zeros((len(lines), 300))
words = []
for i, line in enumerate(lines):
    word_weights = line.split()
    words.append(word_weights[0])
    weight = word_weights[1:]
    glove_weights[i] = np.array([float(w) for w in weight])
word_vocab = [w.decode("utf-8") for w in words]

word2glove = dict(zip(word_vocab, glove_weights))

In [ ]:
u_words_total = [k for k,v in u_words_counter]
word_vocab = dict(zip(word_vocab, range(len(word_vocab))))
word_in_glove = np.array([w in word_vocab for w in u_words_total])

words_in_glove = [w for w,is_true in zip(u_words_total,word_in_glove) if is_true]
words_not_in_glove = [w for w,is_true in zip(u_words_total,word_in_glove) if not is_true]

In [ ]:
df= df[df['Headline'].notnull()]

In [ ]:
word2num = dict(zip(words_in_glove,range(len(words_in_glove))))
len_glove_words = len(word2num)
freq_words_not_glove = [w for w in words_not_in_glove if w in u_words_frequent]
b = dict(zip(freq_words_not_glove,range(len(word2num), len(word2num)+len(freq_words_not_glove))))
word2num = dict(**word2num, **b)
word2num['<Other>'] = len(word2num)
num2word = dict(zip(word2num.values(), word2num.keys()))
df_new = df[df['Headline'].notnull()]
int_text = [[word2num[word] if word in word2num else word2num['<Other>'] 
             for word in content.split()]  for content in df.Headline.values
           ]

In [ ]:
num2word[len(word2num)] = '<PAD>'
word2num['<PAD>'] = len(word2num)

for i, t in enumerate(int_text):
    if len(t)<500:
        int_text[i] = [word2num['<PAD>']]*(500-len(t)) + t
    elif len(t)>500:
        int_text[i] = t[:500]
    else:
        continue

x = np.array(int_text)
y = (df.Label.values=='Fake').astype('int')
y = (df.Label.values=='Authentic').astype('int')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

In [ ]:
from scipy.stats import logistic
X_train=logistic.cdf(X_train)

In [ ]:
x[1:2]

array([[2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213,
        2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213,
        2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213,
        2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213,
        2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213,
        2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213,
        2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213,
        2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213,
        2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213,
        2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213,
        2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213,
        2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213,
        2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213, 2213,
        2213, 2213, 2213, 2213, 2213, 

In [ ]:
import gensim
from gensim.models import KeyedVectors

word2vec = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/Research/NLP/GloveTexFiles/GoogleNewsvectorsnegative300.bin", binary=True)
import pandas as pd
import numpy as np
data=pd.read_excel("/content/drive/MyDrive/Research/NLP/Sentiment Data/Copy of FakevsAuthentic news edit sany.xlsx")
data.Headline=data.Headline.astype(str)
data.Label = data.Label.replace({'Authentic': 1, 'Fake': 0})
data = data.dropna(subset=['Label'])
textdata=data['Headline']
labels=data['Label']
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
MAX_NB_WORDS=200000


tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(textdata)
sequences = tokenizer.texts_to_sequences(textdata)

word_index = tokenizer.word_index

print('Found %s unique tokens.' % len(word_index))

x_traindata = pad_sequences(sequences, maxlen=200)

y_labels = to_categorical(np.asarray(labels))

model=word2vec
vocab_size = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((vocab_size, 300))
for word, i in tokenizer.word_index.items():
	embedding_vector = model.wv[model.wv.index2word[i]]
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

Found 4009 unique tokens.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [ ]:
model = Sequential()
model.add(Embedding(len(word2num), 50)) # , batch_size=batch_size
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2)
         )
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.summary()

Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_30 (Embedding)    (None, None, 50)          110700    
                                                                 
 lstm_12 (LSTM)              (None, 128)               91648     
                                                                 
 dense_12 (Dense)            (None, 1)                 129       
                                                                 
Total params: 202,477
Trainable params: 202,477
Non-trainable params: 0
_________________________________________________________________


In [ ]:
batch_size = 50

model.fit(X_train, y_train, batch_size=batch_size, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
112/112 [==============================] - 175s 2s/step - loss: 0.6893 - accuracy: 0.5426 - val_loss: 0.6944 - val_accuracy: 0.4984
Epoch 2/10
112/112 [==============================] - 179s 2s/step - loss: 0.7605 - accuracy: 0.5279 - val_loss: 0.7126 - val_accuracy: 0.5016
Epoch 3/10
112/112 [==============================] - 172s 2s/step - loss: 0.6973 - accuracy: 0.5100 - val_loss: 0.6969 - val_accuracy: 0.4984
Epoch 4/10
112/112 [==============================] - 169s 2s/step - loss: 0.6917 - accuracy: 0.5295 - val_loss: 0.7067 - val_accuracy: 0.4984
Epoch 5/10
112/112 [==============================] - 169s 2s/step - loss: 0.6791 - accuracy: 0.5627 - val_loss: 0.7142 - val_accuracy: 0.4968
Epoch 6/10
112/112 [==============================] - 169s 2s/step - loss: 0.6902 - accuracy: 0.5311 - val_loss: 0.7389 - val_accuracy: 0.4984
Epoch 7/10
112/112 [==============================] - 169s 2s/step - loss: 0.6720 - accuracy: 0.5927 - val_loss: 0.7866 - val_accuracy: 0.4984

In [ ]:
model.evaluate(X_test, y_test)

20/20 [==============================] - 3s 159ms/step - loss: 0.6912 - accuracy: 0.5210


[0.6912419199943542, 0.5209677219390869]

In [ ]:
sentence = "যুক্তরাষ্ট্রে ঘাস কাটছেন সাকিব আল হাসান"
sentence_num = [word2num[w] if w in word2num else word2num['<Other>'] for w in sentence.split()]
sentence_num = [word2num['<PAD>']]*(500-len(sentence_num)) + sentence_num

sentence_num = np.array(sentence_num)
sentence_num=logistic.cdf(sentence_num)
model.predict(sentence_num[None,:])

array([[0.565838]], dtype=float32)